In [1]:
 #Colab / Jupyter single cell

!pip install transformers accelerate gradio text-to-speech

from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
import torch
import gradio as gr
from text_to_speech import save as tts_save
import uuid

# Load the IBM Granite model from Hugging Face
model_name = "ibm-granite/granite-3.3-2b-instruct"  # as you specified
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16 if device == "cuda" else torch.float32,
)

def rewrite_with_tone(text, tone):
    """
    Rewrites `text` in the chosen `tone` using Granite model.
    We'll build a simple instruction prompt.
    """
    # Construct prompt
    prompt = (
        f"You are a helpful assistant. Rewrite the following text in a {tone} tone, "
        "while preserving its original meaning:\n\n"
        f"{text}\n\nRewritten:"
    )
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    set_seed(42)
    outputs = model.generate(
        **inputs,
        max_new_tokens= max(200, len(text) // 2),  # adjust as needed
        temperature=0.7,
        top_p=0.9,
    )
    rewritten = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Optionally remove the prompt part from rewritten (simple heuristic)
    # But because Granite uses an instruction-format, rewritten may directly be the answer
    return rewritten

def text_to_audio(text):
    """
    Converts text to audio. Uses `text-to-speech` library.
    Returns path to an mp3 file.
    """
    # Generate a random filename
    fname = f"/content/{uuid.uuid4().hex}.mp3"
    # Save TTS
    tts_save(text, "en", file=fname)
    return fname

def process(text, tone):
    if not text.strip():
        return "Please enter some text.", None, None
    # 1. Rewrite
    rewritten = rewrite_with_tone(text, tone)
    # 2. Convert to audio
    audio_path = text_to_audio(rewritten)
    # 3. For Gradio: return rewritten text, and audio file path
    return rewritten, "Here is your audio:", audio_path

# Build Gradio interface
with gr.Blocks(css="""
.gradio-container {
        background: #fff9c4;  /* light yellow */
    }
    .header {
        color: #000000;        /* black text color */

        text-align: center;
    }
 """) as demo:
    gr.Markdown("<h1 class='header'>📖🎙️ EchoVerse: AI Audiobook Generator🎧✨</h1>")
    inp = gr.Textbox(label="📝Input Text", lines=10, placeholder="💡Paste your text here …")
    tone = gr.Radio(["Neutral", "Suspenseful", "Inspiring"], label="Choose Tone", value="Neutral")
    btn = gr.Button("🎶Generate")
    out_text = gr.Textbox(label="✍️Rewritten Text", lines=10)
    out_audio = gr.Audio(label="🗣️Generated Audio", format="mp3")

    btn.click(fn=process, inputs=[inp, tone], outputs=[out_text, gr.Textbox(), out_audio])



demo.launch(share=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.0
    Uninstalling click-8.3.0:
      Successfully uninstalled click-8.3.0


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7c949146667ccbc08f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from huggingface_hub import login

login()  # It will ask you to paste your Hugging Face token
